In [ ]:
# Imports
import os
import re
import json
import subprocess
from pathlib import Path

### Prepare a new Git repository

In [ ]:
dirname = Path("output")

In [ ]:
dirname.mkdir(parents=True, exist_ok=True)

In [ ]:
# Make Git ignore the global configuration files
os.environ['GIT_CONFIG_SYSTEM'] = "/dev/null"
os.environ['GIT_CONFIG_GLOBAL'] = "/dev/null"

In [ ]:
subprocess.run(("git", "init", "--initial-branch=main"), cwd=dirname, check=True)

In [ ]:
_ = subprocess.run(("git", "config", "user.name", "AI-Nix-Packager"), cwd=dirname, check=True)
_ = subprocess.run(("git", "config", "user.email", ""), cwd=dirname, check=True)

### Initialize a new Git branch

In [ ]:
# TODO: don't hardcode
#project_name = "findtow" # https://git.sr.ht/~martijnbraam/findtow
#project_name = "gResistor" # https://github.com/stethewwolf/gResistor
project_name = "postmarketos-tweaks" # https://gitlab.postmarketos.org/postmarketOS/postmarketos-tweaks

In [ ]:
_ = subprocess.run(("git", "checkout", "--orphan", project_name), cwd=dirname, check=True)
_ = subprocess.run(("git", "rm", "-rf", "."), cwd=dirname, check=False) # Ignore failures that happen when there are no files

### Generate the default.nix file using nix-init

In [ ]:
# Now please run nix-init inside of the output directory
nix_file = dirname / "default.nix"
if not os.path.isfile(nix_file):
    if not os.path.isfile("default.nix"):
        raise Exception("default.nix not found, please run nix-init inside of the output directory.")
    else:
        raise Exception("Make sure to run nix-init inside of the output directory, not inside the root of the project!")

In [ ]:
def git_commit(message):
    subprocess.run(("git", "add", "--all"), cwd=dirname, check=True)
    subprocess.run(("git", "commit", "-m", message), cwd=dirname, check=True)

In [ ]:
git_commit("Run nix-init")

In [ ]:
# TODO: run nix-format after every step

In [ ]:
# TODO: pin Nixpkgs revision to the one used by this flake
def nix_build():
    return subprocess.run(
        (
            "nix-build",
            "--no-out-link",
            "--log-format", "internal-json",
             "-E", "(import <nixpkgs> { }).callPackage ./. { }"
        ),
        capture_output=True,
        encoding='utf-8',
        cwd=dirname,
        check=False,
        env={
            "PATH": os.getenv("PATH"),
            "NIX_PATH": os.getenv("NIX_PATH"),
            "NO_COLOR": "", # TODO: debug Nix why this has no effect when --log-format is internal-json
            "NIXPKGS_ALLOW_UNFREE": "1",
        },
    )

In [ ]:
build_result = nix_build()

In [ ]:
# Regex from https://stackoverflow.com/a/14693789
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

def parse_error_from_nix_output(output):
    error_messages = []
    for line in output.split("\n"):
        if line == "":
            continue
        prefix = "@nix "
        assert line.startswith(prefix), f"Line does not start with @nix : {line}"
        line = line.lstrip(prefix)
        parsed = json.loads(line)
        assert "action" in parsed, f"Nix JSON output does not have an action key: {parsed}"
        if parsed["action"] != "msg":
            continue
        assert "msg" in parsed, f"Nix JSON output does not have a msg key: {parsed}"
        if parsed["msg"].startswith("fetching path input "):
            continue
        if "raw_msg" in parsed:
            error_message = parsed["raw_msg"]
            derivation_path_search = re.search('For full logs, run:\n  nix log (/nix/store/.*.drv)', error_message)
            if derivation_path_search:
                derivation_path = derivation_path_search.group(1)
                # TODO: teach nix log to respect the NO_COLOR environment variable
                error_message = subprocess.run(("nix", "log", derivation_path), capture_output=True, encoding='utf-8', cwd=dirname, check=True).stdout
            error_message = ansi_escape.sub('', error_message)
            error_messages.append(error_message)
    num_error_messages = len(error_messages)
    assert num_error_messages == 1, f"Unexpected number of error messages. Expected 1, found {num_error_messages}"
    return error_messages[0]

In [ ]:
# Analyze and filter output

if build_result.returncode == 0:
    print("The nix-build command succeeded 🎉")
else:
    print("The nix-build command failed, let's see what the error was.")
    print(build_result.stdout)
    error_message = parse_error_from_nix_output(build_result.stderr)
    print(error_message)

In [ ]:
# TODO: find instances of the absolute path to the current directory and rewrite them
# TODO: special-case errors like evaluation aborted with the following error message:
# 'lib.customisation.callPackageWith: Function called without required argument "libblkid" at /home/u/Dokumente/repos/RAG-with-Langchain/output/default.nix:8'
# to provide some help to the LLM
# TODO: validate syntax and help the model if it is wrong
# TODO: detect if the model ever outputs the same thing twice and tell the model
# TODO: detect if the error message is the same as before and tell the model that the suggestion did not fix the error
# TODO: take existing derivations from Nixpkgs and modify them, e.g. by deleting lines. Then fine-tune an LLM with the error messages
# TODO: prompt the model to tell if thinks that a dependency is missing and then use something like search.nixos.org but lokal (maybe nix-index) to search for it, then tell the model which dependency to add
# TODO: Run multiple different models at each step and compare
# TODO: detect changes that the LLM should not make, such as changing the hash and revert them
# TODO: if the model does not support reasoning, first ask the model for suggestions on fixing the error, then asking it to implement those suggestions